# Lab 3.1: Sentinel-2 Data Download from Copernicus Dataspace

**Part of the Iceland ML Course: Sentinel-2 Classification Project**

This notebook demonstrates how to acquire and download Sentinel-2 satellite imagery using the Copernicus Dataspace Ecosystem API for land classification tasks.

---

## Project Overview

| Lab | Topic | Status |
|-----|-----------|--------|
| Lab 1 | HPC Access Setup | ✅ Previous |
| Lab 2 | Jupyter-JSC & Git | ✅ Previous |
| Lab 3.1 | **Data Download (Copernicus API)** | 🔄 **Current** |
| Lab 3.2 | Data Preprocessing & Patch Extraction | ⬜ Next |
| Lab 4 | Understanding Transformers | ⬜ Next |
| Lab 4.1 | Training on Sentinel-2 Data | ⬜ Next |
| Lab 5 | Distributed Training (Multi-GPU) | ⬜ Next |
| Lab 6 | Validation & Performance Metrics | ⬜ Next |
| Lab 7 | Foundation Models & TerraToRCH | ⬜ Final |

---

## What You'll Learn

By the end of this lab, you will:
- Register for Copernicus Dataspace access
- Authenticate with the Copernicus Dataspace API
- Search for Sentinel-2 Level 2A imagery by date, location, and cloud cover
- Download Sentinel-2 SAFE archives
- Organize downloaded data for preprocessing

## Quick Start

This lab focuses on the **Data Acquisition** phase of the project pipeline:

```
Data Download from Copernicus (Lab 3.1) ← You are here
    ↓
Data Preprocessing & Patch Extraction (Lab 3.2)
    ↓
Model Training (Lab 4+)
```

---

## Overview

This lab focuses on:
1. **Copernicus Dataspace Registration**: Setting up your account and credentials
2. **API Authentication**: Using OAuth2 to access the API
3. **Searching Sentinel-2 Data**: Finding imagery by date, location, and quality
4. **Downloading SAFE Archives**: Getting full Sentinel-2 products
5. **Data Organization**: Structuring files for Lab 3.2 preprocessing

## Part 1: Setup and Registration

First, you need to register for Copernicus Dataspace access and set up authentication credentials.

### Step 1: Register and Create OAuth2 Client

**Important**: The Copernicus Dataspace uses OAuth2 authentication. Follow these steps:

1. Go to **Copernicus Browser Dashboard**: https://browser.dataspace.copernicus.eu/
2. Login to your account (create one if needed)
3. Click your profile (top right) → **User Settings**
4. Locate **"OAuth clients"** section
5. Click **"Create"** to register a new OAuth client
6. **Give it a name** (e.g., "Iceland-ML-Lab")
7. Set expiration or select "Never expire" (recommended for HPC)
8. Click **"Create"**
9. Copy your **Client ID** and **Client Secret** and save them securely

⚠️ **Security Note**: Your client secret won't be shown again! Save it immediately.

**Your credentials:**
- **Client ID**: `sh-....`
- **Client Secret**: `<YOUR SECRET>`

In [ ]:
import requests
import os
import json
from datetime import datetime
import zipfile
from pathlib import Path

# For visualization later
import matplotlib.pyplot as plt

### Step 2: Configure OAuth2 Client Credentials

OAuth2 Client Credentials is the recommended authentication method for server-to-server communication and HPC jobs. Store credentials securely as environment variables.

In [ ]:
# Configure your Copernicus Dataspace OAuth2 Client Credentials
# Register OAuth client in Dashboard: https://browser.dataspace.copernicus.eu/

# Option 1: Set as environment variables (recommended for HPC)
# export COPERNICUS_CLIENT_ID="your_client_id"
# export COPERNICUS_CLIENT_SECRET="your_client_secret"

# Option 2: Direct assignment (for testing only - DO NOT COMMIT TO GIT!)
COPERNICUS_CLIENT_ID = os.getenv('COPERNICUS_CLIENT_ID', 'sh-...')
COPERNICUS_CLIENT_SECRET = os.getenv('COPERNICUS_CLIENT_SECRET', '<Your Secret>')

# Copernicus Dataspace API endpoints
AUTH_URL = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
SEARCH_URL = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
DOWNLOAD_URL = "https://zipper.dataspace.copernicus.eu/odata/v1/Products"

print("✓ Credentials configured")

### Step 3: Set Environment Variables (Optional)

Store your credentials as environment variables instead of hardcoding them.

**On your HPC system:**

```bash
# Add to ~/.bashrc or ~/.bash_profile
export COPERNICUS_CLIENT_ID="<Your ID>"
export COPERNICUS_CLIENT_SECRET="<Your Secret>"

# Then reload
source ~/.bashrc
```

**Verify setup:**

```bash
# Check variables are set
echo $COPERNICUS_CLIENT_ID
echo $COPERNICUS_CLIENT_SECRET
```

In [ ]:
def get_access_token(client_id, client_secret):
    """
    Get OAuth2 access token from Copernicus Dataspace using Client Credentials flow.
    
    This is the recommended method for server-to-server authentication and HPC jobs.
    
    Parameters:
    -----------
    client_id : str
        OAuth2 Client ID (starts with 'sh-')
    client_secret : str
        OAuth2 Client Secret
    
    Returns:
    --------
    str : Access token if successful, None otherwise
    """
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }
    
    try:
        response = requests.post(AUTH_URL, data=data, timeout=30)
        response.raise_for_status()
        token_data = response.json()
        
        # Extract token and expiration
        access_token = token_data["access_token"]
        expires_in = token_data.get("expires_in", 3600)
        
        print(f"✓ Token obtained (valid for {expires_in//60} minutes)")
        return access_token
        
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            print("❌ Authentication failed: Invalid credentials")
            print("   ✗ Check your CLIENT_ID and CLIENT_SECRET")
            print("   ✗ CLIENT_ID should start with 'sh-'")
        else:
            print(f"❌ HTTP {e.response.status_code}: {e.response.text}")
        return None
        
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        return None

# Get access token
print("Authenticating with Copernicus Dataspace...")
access_token = get_access_token(COPERNICUS_CLIENT_ID, COPERNICUS_CLIENT_SECRET)

if access_token:
    print("✓ Successfully authenticated")
    headers = {"Authorization": f"Bearer {access_token}"}
else:
    print("❌ Authentication failed.")
    print("\nTroubleshooting:")
    print("1. Check environment variables are set:")
    print(f"   COPERNICUS_CLIENT_ID = {COPERNICUS_CLIENT_ID[:15]}...")
    print(f"   COPERNICUS_CLIENT_SECRET = {COPERNICUS_CLIENT_SECRET[:15]}...")
    print("2. Verify credentials in Copernicus Dashboard")
    print("3. See COPERNICUS_SETUP.md for detailed instructions")
    headers = None

---

## Part 2: Define Search Parameters

Specify your study area, date range, and quality criteria for Sentinel-2 imagery.

### Define Region of Interest and Date Range

In [ ]:
# Define your Region of Interest (ROI) as a bounding box
# Format: POLYGON((lon lat, lon lat, ...))
# Example: Bavarian region (Germany) - has CORINE coverage
# Note: You will be selecting an MGRS tile within this region

# Bounding box coordinates [min_lon, min_lat, max_lon, max_lat]
# Bavarian region (Central Europe)
min_lon, min_lat = 8.0, 47.0
max_lon, max_lat = 14.0, 49.5

# Create WKT POLYGON for API query
roi_polygon = f"POLYGON(({min_lon} {min_lat},{max_lon} {min_lat},{max_lon} {max_lat},{min_lon} {max_lat},{min_lon} {min_lat}))"

# Define date range
start_date = '2018-03-01T00:00:00.000Z'
end_date = '2018-10-31T23:59:59.999Z'

# Maximum cloud cover percentage (30% to get good data availability)
max_cloud_cover = 30

print(f"Search Parameters:")
print(f"  Region: Bavarian region (Central Europe with CORINE coverage)")
print(f"  Bounding Box: ({min_lon}, {min_lat}) to ({max_lon}, {max_lat})")
print(f"  Date Range: {start_date[:10]} to {end_date[:10]}")
print(f"  Max Cloud Cover: {max_cloud_cover}%")
print(f"\nNote: Sentinel-2 divides the globe into MGRS tiles (100×100 km each).")
print(f"Your search will find all tiles intersecting this region.")

---

## Part 3: Search for Sentinel-2 Data

Query the Copernicus Dataspace catalog for Sentinel-2 Level 2A imagery matching your criteria.

### Search Sentinel-2 Collection

In [ ]:
def search_sentinel2(start_date, end_date, roi_polygon, max_cloud_cover):
    """
    Search for Sentinel-2 L2A products in Copernicus Dataspace
    """
    # Build OData filter query
    filters = [
        f"Collection/Name eq 'SENTINEL-2'",
        f"Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'S2MSI2A')",
        f"ContentDate/Start gt {start_date}",
        f"ContentDate/Start lt {end_date}",
        f"OData.CSC.Intersects(area=geography'SRID=4326;{roi_polygon}')",
        f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value lt {max_cloud_cover})"
    ]
    
    filter_query = " and ".join(filters)
    
    params = {
        "$filter": filter_query,
        "$orderby": "ContentDate/Start asc",
        "$top": 1000  # Increased to get full date range across all tiles
    }
    
    try:
        response = requests.get(SEARCH_URL, params=params, timeout=60)
        response.raise_for_status()
        results = response.json()
        return results.get('value', [])
    except Exception as e:
        print(f"❌ Search failed: {e}")
        return []

print("✓ Search function defined")

In [ ]:
print("Searching for Sentinel-2 products...")
products = search_sentinel2(start_date, end_date, roi_polygon, max_cloud_cover)

print(f"\n✓ Found {len(products)} Sentinel-2 L2A products")
print(f"✓ All products have <{max_cloud_cover}% cloud cover (filtered server-side)")
print(f"\nThese products span multiple MGRS tiles over your region.")
print(f"Select one MGRS tile and download ~4 acquisitions.\n")
print(f"First 5 products:")
for i, product in enumerate(products[:5]):
    name = product.get('Name', 'Unknown')
    date = product.get('ContentDate', {}).get('Start', 'N/A')[:10]
    size = product.get('ContentLength', 0) / (1024**3)
    
    print(f"  {i+1}. {name}")
    print(f"     Date: {date}, Size: {size:.2f} GB")

### Group Products by MGRS Tile

Before downloading, let's organize products by MGRS tile to ensure we download multiple acquisitions of the same tile.

In [ ]:
# Group products by MGRS tile
from collections import defaultdict

tiles = defaultdict(list)
for product in products:
    product_name = product.get('Name', '')
    # Extract MGRS tile from product name (e.g., T32UPD from S2A_MSIL2A_..._T32UPD_...)
    tile_id = product_name.split('_')[5] if len(product_name.split('_')) > 5 else 'Unknown'
    tiles[tile_id].append(product)

# Display available tiles and their acquisition counts
print("Available MGRS Tiles and Acquisition Counts:")
print("=" * 50)
for tile_id, tile_products in sorted(tiles.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"\nTile {tile_id}: {len(tile_products)} acquisitions")
    for i, product in enumerate(tile_products[:5]):  # Show first 5
        date = product.get('ContentDate', {}).get('Start', 'N/A')[:10]
        size = product.get('ContentLength', 0) / (1024**3)
        print(f"  {i+1}. {date} - {size:.2f} GB")
    if len(tile_products) > 5:
        print(f"  ... and {len(tile_products) - 5} more")

print("\n" + "=" * 50)
print(f"\nRecommendation: Choose a tile with 4+ acquisitions for training data diversity.")

In [ ]:
# Select a tile to work with (choose the one with most acquisitions, or specify manually)
# Option 1: Automatic - select tile with most acquisitions
selected_tile = max(tiles.items(), key=lambda x: len(x[1]))[0] if tiles else None

# Option 2: Manual selection - uncomment and specify tile ID
# selected_tile = "T32UPD"  # Replace with your chosen tile

if selected_tile:
    tile_products = tiles[selected_tile]
    num_acquisitions = len(tile_products)
    
    print(f"Selected MGRS Tile: {selected_tile}")
    print(f"Total acquisitions available: {num_acquisitions}")
    
    # Select 4 evenly spaced acquisitions for temporal diversity
    num_to_select = 4
    if num_acquisitions >= num_to_select:
        # Calculate indices for evenly spaced selection
        indices = [int(i * (num_acquisitions - 1) / (num_to_select - 1)) for i in range(num_to_select)]
        selected_products = [tile_products[i] for i in indices]
    else:
        # If fewer than 4 acquisitions, use all of them
        selected_products = tile_products
        indices = list(range(len(tile_products)))
    
    print(f"\nSelected {len(selected_products)} evenly-spaced acquisitions for temporal diversity:")
    print("=" * 70)
    for i, (idx, product) in enumerate(zip(indices, selected_products)):
        name = product.get('Name', 'Unknown')
        date = product.get('ContentDate', {}).get('Start', 'N/A')[:10]
        size = product.get('ContentLength', 0) / (1024**3)
        print(f"  {i+1}. [{idx+1}/{num_acquisitions}] {date} - {size:.2f} GB")
        print(f"      {name}")
    print("=" * 70)
    print("\nThese acquisitions span the full date range for better training data diversity.")
else:
    print("❌ No tiles found. Adjust your search parameters.")
    selected_products = []

---

## Part 4: Download Sentinel-2 Products

Download selected Sentinel-2 SAFE archives to your local or HPC storage.

### Download Function

In [ ]:
def download_product(product, output_dir, access_token):
    """
    Download a Sentinel-2 product from Copernicus Dataspace
    
    Parameters:
    -----------
    product : dict
        Product metadata from search results
    output_dir : str
        Directory to save downloaded file
    access_token : str
        OAuth2 access token
    
    Returns:
    --------
    str : Path to downloaded file, or None if failed
    """
    product_id = product['Id']
    product_name = product['Name']
    
    # Create download directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Output file path
    output_file = os.path.join(output_dir, f"{product_name}.zip")
    
    # Check if already downloaded
    if os.path.exists(output_file):
        print(f"⚠ File already exists: {product_name}.zip")
        return output_file
    
    # Build download URL
    download_url = f"{DOWNLOAD_URL}({product_id})/$value"
    
    headers = {"Authorization": f"Bearer {access_token}"}
    
    try:
        print(f"Downloading: {product_name}")
        print(f"  Size: {product['ContentLength'] / (1024**3):.2f} GB")
        
        # Stream download with progress
        with requests.get(download_url, headers=headers, stream=True, timeout=300) as response:
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            block_size = 8192
            downloaded = 0
            
            with open(output_file, 'wb') as f:
                for chunk in response.iter_content(chunk_size=block_size):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        
                        # Print progress every 100 MB
                        if downloaded % (100 * 1024 * 1024) < block_size:
                            progress = (downloaded / total_size) * 100 if total_size > 0 else 0
                            print(f"  Progress: {progress:.1f}% ({downloaded / (1024**3):.2f} GB)")
        
        print(f"✓ Download complete: {output_file}")
        return output_file
        
    except Exception as e:
        print(f"❌ Download failed: {e}")
        # Clean up partial download
        if os.path.exists(output_file):
            os.remove(output_file)
        return None

print("✓ Download function defined")

### Download A Selected Acquisition

In [ ]:
# Define download directory (modify for your HPC storage)
download_dir = "/p/scratch/training2600/hashim1/data"

# Download the selected product
if products and access_token:
    downloaded_file = download_product(products[0], download_dir, access_token)
    if downloaded_file:
        print(f"\n✓ Product saved to: {downloaded_file}")
else:
    print("❌ Cannot download: No products found or authentication failed")

### Download Multiple Acquisitions

In [ ]:
# Download the evenly-spaced acquisitions from the selected tile
download_dir = "/p/scratch/training2600/hashim1/data"

if selected_tile and access_token and selected_products:
    print(f"Downloading {len(selected_products)} evenly-spaced acquisitions from tile {selected_tile}...")
    print("=" * 60)
    
    for i, product in enumerate(selected_products):
        date = product.get('ContentDate', {}).get('Start', 'N/A')[:10]
        print(f"\n--- Downloading {i+1}/{len(selected_products)} ({date}) ---")
        download_product(product, download_dir, access_token)
    
    print("\n" + "=" * 60)
    print(f"✓ Downloaded {len(selected_products)} acquisitions from tile {selected_tile}")
    print(f"✓ Acquisitions are evenly spaced across the time range")
    print(f"✓ All files saved to: {download_dir}")
else:
    print("❌ Cannot download: No tile/products selected or authentication failed")

### Visualisation

In [ ]:
import numpy as np
import glob

def extract_and_visualize_sentinel2(zip_path, output_dir=None):
    """
    Extract a Sentinel-2 ZIP file and create an RGB visualization.
    
    Parameters:
    -----------
    zip_path : str
        Path to the Sentinel-2 ZIP file
    output_dir : str, optional
        Directory to extract to. If None, extracts to same directory as ZIP.
    
    Returns:
    --------
    str : Path to extracted SAFE directory
    """
    import zipfile
    import rasterio
    from rasterio.plot import show
    
    if output_dir is None:
        output_dir = os.path.dirname(zip_path)
    
    # Extract ZIP file
    print(f"Extracting: {os.path.basename(zip_path)}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get the SAFE directory name (first item in the archive)
        safe_dir = zip_ref.namelist()[0].split('/')[0]
        safe_path = os.path.join(output_dir, safe_dir)
        
        if os.path.exists(safe_path):
            print(f"⚠ Already extracted: {safe_dir}")
        else:
            zip_ref.extractall(output_dir)
            print(f"✓ Extracted to: {safe_path}")
    
    return safe_path


def visualize_sentinel2_rgb(safe_path, figsize=(12, 12)):
    """
    Create an RGB visualization from Sentinel-2 SAFE directory.
    
    Uses bands B04 (Red), B03 (Green), B02 (Blue) at 10m resolution.
    
    Parameters:
    -----------
    safe_path : str
        Path to the extracted .SAFE directory
    figsize : tuple
        Figure size for the plot
    """
    import rasterio
    from rasterio.enums import Resampling
    
    # Find the 10m resolution bands (B02, B03, B04)
    # Path pattern: .SAFE/GRANULE/*/IMG_DATA/R10m/*_B0X_10m.jp2
    granule_path = os.path.join(safe_path, 'GRANULE')
    
    if not os.path.exists(granule_path):
        print(f"❌ GRANULE directory not found in {safe_path}")
        return
    
    # Get the tile subdirectory
    tile_dirs = [d for d in os.listdir(granule_path) if os.path.isdir(os.path.join(granule_path, d))]
    if not tile_dirs:
        print("❌ No tile directories found")
        return
    
    tile_dir = os.path.join(granule_path, tile_dirs[0])
    
    # Try R10m directory first (newer format), then IMG_DATA (older format)
    r10m_path = os.path.join(tile_dir, 'IMG_DATA', 'R10m')
    img_data_path = os.path.join(tile_dir, 'IMG_DATA')
    
    if os.path.exists(r10m_path):
        band_dir = r10m_path
        band_pattern = '*_B0{}_10m.jp2'
    else:
        band_dir = img_data_path
        band_pattern = '*_B0{}.jp2'
    
    # Find band files
    bands = {}
    for band_num in ['2', '3', '4']:
        pattern = os.path.join(band_dir, band_pattern.format(band_num))
        matches = glob.glob(pattern)
        if matches:
            bands[f'B0{band_num}'] = matches[0]
        else:
            # Try alternative pattern for older format
            alt_pattern = os.path.join(band_dir, f'*B0{band_num}*.jp2')
            alt_matches = glob.glob(alt_pattern)
            if alt_matches:
                bands[f'B0{band_num}'] = alt_matches[0]
    
    if len(bands) < 3:
        print(f"❌ Could not find all RGB bands. Found: {list(bands.keys())}")
        print(f"   Searched in: {band_dir}")
        return
    
    print(f"✓ Found bands: {list(bands.keys())}")
    
    # Read bands with downsampling for visualization (full resolution can be huge)
    downsample_factor = 10  # Read at 1/10 resolution for faster display
    
    rgb_bands = []
    for band_name in ['B04', 'B03', 'B02']:  # RGB order
        with rasterio.open(bands[band_name]) as src:
            # Calculate new dimensions
            new_height = src.height // downsample_factor
            new_width = src.width // downsample_factor
            
            # Read with resampling
            band_data = src.read(
                1,
                out_shape=(new_height, new_width),
                resampling=Resampling.average
            )
            rgb_bands.append(band_data)
    
    # Stack into RGB array
    rgb = np.stack(rgb_bands, axis=-1)
    
    # Normalize for visualization (typical Sentinel-2 values range 0-10000)
    # Use percentile-based stretching for better visualization
    p2, p98 = np.percentile(rgb[rgb > 0], (2, 98))
    rgb_normalized = np.clip((rgb - p2) / (p98 - p2), 0, 1)
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    
    # RGB composite
    axes[0].imshow(rgb_normalized)
    axes[0].set_title('Sentinel-2 RGB Composite (B4-B3-B2)', fontsize=12)
    axes[0].axis('off')
    
    # False color (NIR-Red-Green) if available
    # For now, show a histogram of the data
    axes[1].hist(rgb[:,:,0].flatten()[::100], bins=50, alpha=0.7, label='Red (B04)', color='red')
    axes[1].hist(rgb[:,:,1].flatten()[::100], bins=50, alpha=0.7, label='Green (B03)', color='green')
    axes[1].hist(rgb[:,:,2].flatten()[::100], bins=50, alpha=0.7, label='Blue (B02)', color='blue')
    axes[1].set_xlabel('Reflectance Value')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Band Value Distribution', fontsize=12)
    axes[1].legend()
    axes[1].set_xlim(0, 5000)
    
    plt.tight_layout()
    plt.savefig("S2_vis.png")
    
    # Print some metadata
    with rasterio.open(bands['B04']) as src:
        print(f"\nImage Metadata:")
        print(f"  Original size: {src.width} x {src.height} pixels")
        print(f"  Resolution: {src.res[0]}m x {src.res[1]}m")
        print(f"  CRS: {src.crs}")
        print(f"  Bounds: {src.bounds}")
    
    return rgb_normalized

print("✓ Visualization functions defined")

In [ ]:
# Visualize the first downloaded acquisition
# Adjust the path to match your download location

def is_valid_zipfile(filepath):
    """Check if a file is a valid ZIP file."""
    try:
        with zipfile.ZipFile(filepath, 'r') as zf:
            return zf.testzip() is None
    except (zipfile.BadZipFile, Exception):
        return False

if selected_products and access_token:
    # Get the first downloaded product
    first_product = selected_products[0]
    product_name = first_product.get('Name', '')
    
    # Build possible paths - handle various naming conventions
    # The product name might already end with .SAFE
    base_name = product_name.rstrip('.SAFE') if product_name.endswith('.SAFE') else product_name
    
    possible_paths = [
        os.path.join(download_dir, f"{product_name}"),           # Direct name (if it's a directory)
        os.path.join(download_dir, f"{product_name}.SAFE"),      # With .SAFE suffix
        os.path.join(download_dir, f"{base_name}.SAFE"),         # Base name with .SAFE
        os.path.join(download_dir, f"{product_name}.zip"),       # With .zip suffix  
        os.path.join(download_dir, f"{base_name}.zip"),          # Base name with .zip
    ]
    
    safe_path = None
    zip_path = None
    
    # Find the first existing path
    for path in possible_paths:
        if os.path.exists(path):
            if os.path.isdir(path):
                # It's a directory (SAFE format)
                safe_path = path
                print(f"✓ Found SAFE directory: {os.path.basename(path)}")
                break
            elif path.endswith('.zip') and is_valid_zipfile(path):
                # It's a valid ZIP file
                zip_path = path
                print(f"✓ Found valid ZIP file: {os.path.basename(path)}")
                break
            elif path.endswith('.zip'):
                # File exists but is not a valid ZIP - might be misnamed SAFE dir
                print(f"⚠ Found {os.path.basename(path)} but it's not a valid ZIP file")
                print("  Checking if it might be a SAFE directory saved with wrong extension...")
                # Check if there's a SAFE directory with similar name
                continue
    
    # If we found a ZIP file, extract it
    if zip_path and not safe_path:
        print(f"\nVisualizing: {os.path.basename(zip_path)}")
        print("=" * 60)
        safe_path = extract_and_visualize_sentinel2(zip_path)
    
    # Visualize if we have a SAFE path
    if safe_path and os.path.isdir(safe_path):
        print(f"\nVisualizing: {os.path.basename(safe_path)}")
        print("=" * 60)
        
        # Create RGB visualization directly from SAFE directory
        print("\nCreating RGB visualization...")
        visualize_sentinel2_rgb(safe_path)
        
        print("\n" + "=" * 60)
        print("✓ Visualization complete!")
        print("  - RGB composite shows the natural color view")
        print("  - Histogram shows the distribution of reflectance values")
        print("  - Data is ready for preprocessing in Lab 3.2")
    else:
        # Last resort: try to find any .SAFE directory in download_dir
        safe_dirs = glob.glob(os.path.join(download_dir, "*.SAFE"))
        if safe_dirs:
            safe_path = safe_dirs[0]
            print(f"\nFound SAFE directory: {os.path.basename(safe_path)}")
            print("=" * 60)
            
            print("\nCreating RGB visualization...")
            visualize_sentinel2_rgb(safe_path)
            
            print("\n" + "=" * 60)
            print("✓ Visualization complete!")
        else:
            print(f"\n❌ No valid Sentinel-2 data found in: {download_dir}")
            print(f"\nSearched for:")
            for p in possible_paths[:3]:
                print(f"  - {p}")
            print("\nTip: Check what files exist in your download directory:")
            print(f"  ls -la {download_dir}")
else:
    print("❌ No products selected or authentication failed")
    print("   Run the search and download cells first.")

---

## Part 5: (Optional) Visualize with GEE

If you want to quickly visualize search results before downloading, you can optionally use Google Earth Engine. This is optional and only for reconnaissance.

### Optional: Visualize ROI with GEE

If you want to preview your ROI using Google Earth Engine (optional):

In [ ]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()

# Bounding box coordinates [min_lon, min_lat, max_lon, max_lat]
# Bavarian region (Central Europe)
min_lon, min_lat = 8.0, 47.0
max_lon, max_lat = 14.0, 49.5

# Create ee.Geometry.Rectangle (simpler approach)
roi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# OR create from WKT string:
# roi_polygon = f"POLYGON(({min_lon} {min_lat},{max_lon} {min_lat},{max_lon} {max_lat},{min_lon} {max_lat},{min_lon} {min_lat}))"
# roi = ee.Geometry(roi_polygon, proj='EPSG:4326', geodesic=False)

Map = geemap.Map(center=[48.0, 11.0], zoom=6)
Map.addLayer(roi, {'color': 'FF0000'}, 'ROI')
Map

---

## Part 6: Data Organization

Organize your downloaded data for efficient preprocessing in Lab 3.2.

In [ ]:
# Recommended directory structure for the project
import os

base_dir = "/p/scratch/training2600/YOUR_USERNAME/data"

directory_structure = {
    "sentinel2_data": "Downloaded Sentinel-2 ZIP files",
    "sentinel2_extracted": "Extracted SAFE directories",
    "sentinel2_geotiff": "Converted GeoTIFF files",
    "corine_data": "CORINE land cover maps",
    "training_data": "Preprocessed training patches"
}

print("Recommended Directory Structure:")
print(f"\n{base_dir}/")
for dir_name, description in directory_structure.items():
    dir_path = os.path.join(base_dir, dir_name)
    print(f"  ├── {dir_name}/  # {description}")
    # Uncomment to create directories:
    os.makedirs(dir_path, exist_ok=True)

print("\nTip: Create these directories before starting Lab 3.2")

---

## Summary

This notebook demonstrated how to:

1. **Register for Copernicus Dataspace**: Set up access via the Dataspace Browser
2. **Authenticate via API**: Use OAuth2 to obtain access tokens
3. **Search for Sentinel-2 Data**: Query the catalog by date, location, and cloud cover
4. **Download SAFE Archives**: Retrieve full Sentinel-2 products as ZIP files
5. **Organize Data**: Structure files for efficient preprocessing

The downloaded Sentinel-2 ZIP files are ready for preprocessing in **Lab 3.2**.

---

## What's Next?

### Before Moving to Lab 3.2

**1. Verify Downloads**
   - Check that Sentinel-2 ZIP files are complete (5-10 GB each)
   - Note file paths for Lab 3.2

**2. Extract ZIP Files** (Optional)
   - Extract `.SAFE` directories from ZIP files
   - Or let Lab 3.2 handle extraction automatically

**3. Prepare Storage**
   - Ensure sufficient disk space (~50 GB per tile including extracted data)
   - Create directory structure as shown above

### Next Lab: Lab 3.2 - Data Preprocessing & Patch Extraction

In **Lab 3.2**, you'll:
- Extract Sentinel-2 bands from SAFE format
- Convert data to GeoTIFF for analysis
- Extract 3x3 patches around LUCAS ground truth points
- Create training data for model development

### Data Pipeline Recap

```python
# Lab 3.1 produces:
Sentinel-2 ZIP files (SAFE format)
    ↓
# Lab 3.2 processes:
Extract bands from SAFE archives
Convert to GeoTIFF
Extract patches around ground truth points
    ↓
# Lab 4 trains:
Transformer model on patches
```

---

## Resources & References

- **Copernicus Dataspace Browser**: https://browser.dataspace.copernicus.eu/
- **Copernicus Dataspace API Docs**: https://documentation.dataspace.copernicus.eu/APIs.html
- **Sentinel-2 Product Specification**: https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2
- **OData Protocol**: https://www.odata.org/

---

## Troubleshooting & FAQ

**Q: I can't find the old Copernicus Hub (scihub.copernicus.eu) - where is it?**
- The old SciHub was decommissioned. Use the new **Copernicus Dataspace Ecosystem** at https://dataspace.copernicus.eu/

**Q: How do I get API credentials?**
- Register at https://identity.dataspace.copernicus.eu/
- Use the same credentials for both the browser and API access
- No separate API key needed - OAuth2 tokens are generated on-the-fly

**Q: Download is very slow - what can I do?**
- Use HPC systems with high-bandwidth connections
- Download during off-peak hours
- Consider downloading multiple tiles in parallel with separate jobs

**Q: What if I hit data limits?**
- Copernicus Dataspace has generous free-tier limits
- For large-scale downloads, request increased quotas

**Q: Can I use the browser to download instead of the API?**
- Yes! Use https://browser.dataspace.copernicus.eu/
- Search for products visually
- Download via the web interface
- Better for small numbers of tiles

**Q: How do I know which tile covers my area?**
- Search by coordinates in the Dataspace Browser
- The search results will show the MGRS tile ID

---

**Course Contact**: Refer to course materials for instructor email and office hours  
**Last Updated**: February 2026